In [1]:
import pandas as pd
from twarc import Twarc
import datetime
import time

In [2]:

columns = ["screen_name", "tweet_id","created_at","embeded_url","expanded_url","author","title",
           "content", "day_0_time","day_0_retweet_count"]
columns

['screen_name',
 'tweet_id',
 'created_at',
 'embeded_url',
 'expanded_url',
 'author',
 'title',
 'content',
 'day_0_time',
 'day_0_retweet_count']

## Should we add favorite_count as well?

In [3]:
df = pd.DataFrame(columns=columns)

def export_dataset(df):
    name = "../data/dataset_{}.csv".format(datetime.datetime.now().strftime("%B_%d_%y_%H_%M_%S"))
    df.to_csv(name)

# export_dataset(df)
df.head()

,screen_name,tweet_id,created_at,embeded_url,expanded_url,author,title,content,day_0_time,day_0_retweet_count


In [4]:
consumer_key=''
consumer_secret=''
access_token=''
access_token_secret=''
 
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

a tweet is a valid tweet if
1. it has user\[id\} belonging to the list of listed ids and has embeded url
2. **if has a retweet that satisfies criteria 1 and the source tweet is published today**

In [5]:
# tweets = set()
actually_added = 0
parsed = 0
max_limit = 100
follow_list = [807095,1652541,3108351,51241574]
follow_str = ",".join([str(x) for x in follow_list])

In [6]:
start = time.time()
print("Following users ",follow_str)
for tweet in t.filter(follow=follow_str):
    parsed += 1
    if parsed%100 == 0:
        print("Parsed :",parsed,"\n Added : ",actually_added)
    # tweet for a user we are following
    if 'retweeted_status' not in tweet:
        if tweet['user']['id'] in follow_list \
        and len(tweet['entities']['urls']) != 0 \
        and (df['tweet_id'] != tweet['id']).all():
            print("From source", tweet['id'])
            actually_added += 1
            current = datetime.datetime.now(datetime.timezone.utc)
            df = df.append({'screen_name':tweet['user']['screen_name'], 
                  'tweet_id':tweet['id'],
                  'created_at' : tweet['created_at'],
                  'embeded_url' : tweet['entities']['urls'][0]['url'],
                  'day_0_retweet_count': tweet['retweet_count'],
                  'day_0_time': current}, ignore_index=True)

    else:
        if tweet['retweeted_status']['user']['id'] in follow_list \
        and 'extended_tweet' in tweet['retweeted_status'] \
        and 'entities' in tweet["retweeted_status"]["extended_tweet"] \
        and len(tweet["retweeted_status"]["extended_tweet"]["entities"]['urls']) != 0\
        and (df['tweet_id'] != tweet['retweeted_status']['id']).all():
            created_time = datetime.datetime.strptime(tweet["retweeted_status"]["created_at"],"%a %b %d %H:%M:%S %z %Y")
            current = datetime.datetime.now(datetime.timezone.utc)
            diff =  current - created_time
            print(diff)
            if diff.days == 0:
                print("From retweet", tweet['id'],"adding tweet : ",tweet["retweeted_status"]["id"])
                df = df.append({'screen_name':tweet["retweeted_status"]["user"]["screen_name"], 
                      'tweet_id':tweet["retweeted_status"]["id"],
                      'created_at' : tweet["retweeted_status"]["created_at"],
                      'embeded_url' : tweet["retweeted_status"]["extended_tweet"]["entities"]["urls"][0]["url"],
                      'day_0_retweet_count': tweet['retweeted_status']['retweet_count'],
                      'day_0_time': current}, ignore_index=True)
                actually_added += 1
#             print(tweet['created_at'])
#             print(tweet['id'])
#             print(tweet['user']['id'])
#             print(tweet["retweeted_status"]["id"])
#             print(tweet["retweeted_status"]["created_at"])
#             print(tweet["retweeted_status"]["user"]["screen_name"])
#             print(tweet["retweeted_status"]["extended_tweet"]["entities"]["urls"][0]["url"])
    if actually_added == max_limit:
        break
print("Parsed :",parsed,"\n Added : ",actually_added,"in ", time.time()-start)

Following users  807095,1652541,3108351,51241574
0:12:54.435578
From retweet 1270156882687254533 adding tweet :  1270153658513207296
3 days, 13:39:13.610304
10:33:06.526132
From retweet 1270156941751455744 adding tweet :  1269997639992573954
0:03:14.239241
From retweet 1270156944825868290 adding tweet :  1270156154421690368
1 day, 23:53:12.104558
1:33:24.396740
From retweet 1270156995824431105 adding tweet :  1270133513891840002
4:18:27.888459
From retweet 1270157002002640897 adding tweet :  1270091981738274818
0:08:33.807928
From retweet 1270157026904010753 adding tweet :  1270154896105328640
0:15:52.661265
From retweet 1270157047229808645 adding tweet :  1270153075555172357
4:13:56.862870
From retweet 1270157052313247744 adding tweet :  1270093170479677441
1:53:34.507104
From retweet 1270157054976507904 adding tweet :  1270128497743343628
9:13:36.787590
From retweet 1270157064573202433 adding tweet :  1270017767186534400
13:04:41.626476
From retweet 1270157068054360064 adding tweet :

10:43:46.530358
From retweet 1270165905213882369 adding tweet :  1270003920832782336
6:39:00.051613
From retweet 1270165945147895817 adding tweet :  1270065557530763266
Parsed : 1800 
 Added :  94
From source 1270166130729062403
From source 1270166221644861440
From source 1270166257988501505
2 days, 8:00:16.708996
12:21:03.838575
From retweet 1270166372039811077 adding tweet :  1269979901651795970
4:15:52.776158
From retweet 1270166443112361984 adding tweet :  1270102072239808512
8:08:50.037899
From retweet 1270166515543764993 adding tweet :  1270043519864930305
Parsed : 1886 
 Added :  100 in  2298.345035791397


|Count|Time|
| ------------- |:-------------:| 
|10|123.57|

In [7]:
df.head(max_limit)

,screen_name,tweet_id,created_at,embeded_url,expanded_url,author,title,content,day_0_time,day_0_retweet_count
0,nytimes,1270153658513207296,Tue Jun 09 00:40:05 +0000 2020,https://t.co/Uamr8HXmsO,NaN,NaN,NaN,NaN,2020-06-09 00:52:59.435578+00:00,59
1,nytimes,1269997639992573954,Mon Jun 08 14:20:07 +0000 2020,https://t.co/U5RPu4u2ik,NaN,NaN,NaN,NaN,2020-06-09 00:53:13.526132+00:00,45
2,Reuters,1270156154421690368,Tue Jun 09 00:50:00 +0000 2020,https://t.co/eEGZQHKbIc,NaN,NaN,NaN,NaN,2020-06-09 00:53:14.239241+00:00,17
3,nytimes,1270133513891840002,Mon Jun 08 23:20:02 +0000 2020,https://t.co/XCvQDk49kb,NaN,NaN,NaN,NaN,2020-06-09 00:53:26.396740+00:00,354
4,Reuters,1270091981738274818,Mon Jun 08 20:35:00 +0000 2020,https://t.co/TwDCoKt9No,NaN,NaN,NaN,NaN,2020-06-09 00:53:27.888459+00:00,72
...,...,...,...,...,...,...,...,...,...,...
95,Reuters,1270166221644861440,Tue Jun 09 01:30:00 +0000 2020,https://t.co/0P4vfyeVrp,NaN,NaN,NaN,NaN,2020-06-09 01:30:06.247189+00:00,0
96,WSJ,1270166257988501505,Tue Jun 09 01:30:09 +0000 2020,https://t.co/ULQQjBPOaM,NaN,NaN,NaN,NaN,2020-06-09 01:30:14.669131+00:00,0
97,nytimes,1269979901651795970,Mon Jun 08 13:09:38 +0000 2020,https://t.co/CpoDhki6yS,NaN,NaN,NaN,NaN,2020-06-09 01:30:41.838575+00:00,260
98,WSJ,1270102072239808512,Mon Jun 08 21:15:06 +0000 2020,https://t.co/JpT67k0Ip7,NaN,NaN,NaN,NaN,2020-06-09 01:30:58.776158+00:00,53


In [ ]:
def get_retweet_count(twarc_api, id):
    for tweet in twarc_api.hydrate([id]):
        return tweet['retweet_count']
print(get_retweet_count(t, 1270044909886672896))

In [ ]:
for tweet in t.hydrate([1270148193792204802 ]):
    print(tweet['retweeted_status']['user']['id'] in follow_list)

for tweet in t.hydrate([1270044909886672896 ]):
    print(tweet['user']['id'] in follow_list)
#     print(tweet)
#     print(tweet['user']['screen_name'])
#     print(tweet['id'])
#     print(tweet['created_at'])
#     print(tweet['retweeted_status']['id'])
#     print(tweet['entities']['urls'][0]['url'])
#     print(tweet['retweeted_status'])
#     print(tweet['favorite_count'])
#     print("retweeted_status count", tweet['retweeted_status']['retweet_count'])
#     print(datetime.datetime.now())
#     df = df.append({'screen_name':tweet['user']['screen_name'], 
#               'tweet_id':tweet['id'],
#               'created_at' : tweet['created_at'],
#               'embeded_url' : tweet['entities']['urls'][0]['url'],
#               'day_0_retweet_count': tweet['retweet_count'],
#               'day_0_time':datetime.datetime.now()}, ignore_index=True)
# created_time = datetime.datetime.strptime(tweet['created_at'],"%a %b %d %H:%M:%S %z %Y")
# diff =  datetime.datetime.now(datetime.timezone.utc) - created_time
# abs(diff.days), datetime.datetime.now(datetime.timezone.utc), created_time

In [8]:
export_dataset(df)

In [ ]:
if (df['tweet_id'] == tweet['id']).any():
    print("yes")

In [ ]:
df.head()

In [ ]:
(df['tweet_id'] == tweet['id']).any()

In [3]:
with open('../data/dataset_June_08_20_18_31_56.csv', 'r') as file:
    print(file.mode)

r


In [27]:
import glob
import os.path as path
from operator import itemgetter

In [35]:
files = glob.glob("../data/*.csv")
files_time = [(path.getmtime(file), file) for file in files]
files_time.sort(key=itemgetter(0), reverse=True)
[file for time, file in files_time], files_time[0][1]

(['../data\\dataset_July_08_20_18_31_56.csv',
  '../data\\dataset_June_09_20_05_57_32.csv',
  '../data\\dataset_June_08_20_18_38_22.csv',
  '../data\\dataset_June_08_20_18_31_56.csv'],
 '../data\\dataset_July_08_20_18_31_56.csv')

In [4]:
import pandas as pd
import numpy as np
import time
import threading

In [5]:
def method1():
    global original_df
    while True:
        original_df = original_df.append({'a':1, 'b':2,'c':3}, ignore_index=True)
        print("method 1 : ", len(original_df))
        time.sleep(3)

def method2():
    global original_df
    while True:
        print("method 2 : ", len(original_df))
        time.sleep(4)

def main():
    global original_df
    original_df = pd.DataFrame(columns=['a', 'b', 'c'])
    watcher_t = threading.Thread(target=method1, daemon=True)
    scheduler_t = threading.Thread(target=method2, daemon=True)
    watcher_t.start()
    scheduler_t.start()
    watcher_t.join()
    scheduler_t.join()

In [6]:
main()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "e:\project_viralnews\venv\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "e:\project_viralnews\venv\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-5e8ac31639a7>", line 4, in method1
    df = df.append({'a':1, 'b':2,'c':3}, ignore_index=True)
UnboundLocalError: local variable 'df' referenced before assignment

Exception in thread Thread-7:
Traceback (most recent call last):
  File "e:\project_viralnews\venv\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "e:\project_viralnews\venv\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-5e8ac31639a7>", line 11, in method2
    print("method 2 : ", len(df))
NameError: name 'df' is not defined

